In [ ]:
import os
import argparse
import cv2
import numpy as np
import torch
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from pytorch_grad_cam.ablation_layer import AblationLayerVit

# # 确保结果一致性的随机种子设置
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

# # 设置随机种子
set_seed()

# 选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载预训练的 ViT 模型 (使用 deit_base_patch16_224)
mpath = r'/xxxx.pth'

# 使用 deit_base_patch16_224 模型
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=False)

# 修改分类头，适应 2 个类别
num_classes = 2  # 将类别数量改为 2
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=num_classes)

# 加载预训练权重，但忽略分类头部分的权重加载
state_dict = torch.load(mpath, map_location=device)

# 删除预训练权重中的分类头（head）的参数
del state_dict['head.weight']
del state_dict['head.bias']

# 加载模型参数，忽略 head
model.load_state_dict(state_dict, strict=False)

model.eval()  # 确保模型在评估模式
model = model.to(device)

# 定义 reshape_transform 函数，适用于 ViT 模型
def reshape_transform(tensor, height=14, width=14):
    # 去掉类别标记
    result = tensor[:, 1:, :].reshape(tensor.size(0), height, width, tensor.size(2))
    # 将通道维度放到第一个位置
    result = result.transpose(2, 3).transpose(1, 2)
    return result

# 指定 target_layer 使用最后一个 transformer block 的 norm 层
target_layer = model.blocks[-1].norm1

# 创建 GradCAM 对象，传递 target_layers
cam = GradCAM(model=model, target_layers=[target_layer], reshape_transform=reshape_transform)

# 输入图像路径
image_path = r'/xxxx.png'

# 读取输入图像，OpenCV 加载 BGR 格式，需要转换为 RGB 格式
rgb_img = cv2.imread(image_path, 1)[:, :, ::-1]  # 将BGR转换为RGB
rgb_img = cv2.resize(rgb_img, (224, 224))  # 调整图像大小

# 预处理图像，将图像转换为输入模型的形式
input_tensor = preprocess_image(rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

# 确保输入是 4 维的 (B, C, H, W)
if len(input_tensor.shape) == 3:
    input_tensor = input_tensor.unsqueeze(0).to(device)  # 添加批量维度 (1, C, H, W)

# 计算 Grad-CAM，传递类别作为位置参数
target_category = None  # 可以指定类别（如0, 1），为 None 时表示最高预测类别
grayscale_cam = cam(input_tensor, target_category)

# 将 Grad-CAM 输出的热图叠加到原始图像上
heatmap = cv2.applyColorMap(np.uint8(255 * grayscale_cam[0]), cv2.COLORMAP_JET)

# 显示红色为关注区域，蓝色为不关注区域
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)  # 将 BGR 转换为 RGB，以便红色高亮
visualization = cv2.addWeighted(rgb_img / 255.0, 0.6, heatmap / 255.0, 0.4, 0)

# 定义保存路径
output_dir = r'/results/cam_output'  # 替换为你想保存的文件夹路径

# 确保保存路径存在，如果不存在则创建
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定义保存文件名和完整路径
output_file = os.path.join(output_dir, 'cam_with_red_focus.jpg')

# 保存可视化的结果
cv2.imwrite(output_file, np.uint8(255 * visualization))
print(f'Grad-CAM visualization saved as {output_file}')
